In [ ]:
# データセット読み込みと整形
import load_sms_call_internet_mi

time_unit = 'minute'
CellID = 1

train, test, y_datetime = load_sms_call_internet_mi.load_dataset_ID(time_unit, CellID)

In [ ]:
# xに説明変数を、yに目的変数を格納
train_x = train.drop(columns=['internet'])
train_y = train['internet']
test_x = test.drop(columns=['internet'])
test_y = test['internet']

In [ ]:
# 値を0～1の範囲に正規化し、pandas DataframeからNumPy配列に変更（mms_yは評価指標に使用）
import normalize_train_test
train_x, test_x, train_y, test_y, mms_y = normalize_train_test.apply_MMS(train_x, test_x, train_y, test_y)

In [ ]:
# ["データ数", "timesteps", "特徴量数"]の三次元構造に変換（x_train, x_testに名称変更）
# timesteps時間以降のデータを目的変数に設定（y_train, y_testに名称変更）
import set_timesteps
timesteps = 24*6
x_train, x_test, y_train, y_test = set_timesteps.create_timeseries_data(train_x, test_x, train_y, test_y, timesteps)

In [ ]:
# モデル構築
import build_model
model, history = build_model.build_model_GRU_Optuna(x_train, y_train, timesteps, time_unit)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot') #グラフスタイル

plt.figure(figsize=(15, 8))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Valid Loss')
plt.title('Model Loss (GRU)')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(loc='upper right')

# y軸の上限値を指定（例：0から1.0までの範囲に設定）
plt.ylim(0, 0.1)
plt.show()

In [ ]:
# テストデータに対する予測を行い、評価結果を表示
import evaluate_model_performance
y_test_pred, y_test_true = evaluate_model_performance.evaluate_model(x_test, y_test, mms_y, model)

In [ ]:
from matplotlib.dates import AutoDateLocator, DateFormatter
import pandas as pd
import matplotlib.pyplot as plt

# datetimeの型を確認・変換
datetime = y_datetime[timesteps:].reset_index(drop=True)
datetime = pd.to_datetime(datetime)

# 全体のフォントサイズを設定
plt.rcParams.update({'font.size': 16})

# グラフの描画
plt.figure(figsize=(30, 8))
plt.plot(datetime, y_test_true, linestyle='--', color='blue', label='true')
plt.plot(datetime, y_test_pred, color='orange', label='pred')

# x軸の目盛りを調整
ax = plt.gca() 
locator = AutoDateLocator() 
formatter = DateFormatter("%Y-%m-%d") 
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

# ラベルを回転して見やすくする
plt.xticks(rotation=45)

# その他設定
plt.ylabel('value', labelpad=30)
plt.xlabel('datetime', labelpad=30)
plt.legend(loc='upper right')

plt.ylim(0, 30)
plt.show()